In [1]:
import f3dasm
import json
import inspect
import numpy as np
import pandas as pd

2023-02-27 14:19:58.979118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 14:19:59.130272: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-27 14:19:59.760891: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64:/usr/local/cuda-11.1/lib64
2023-02-27 14:19:59.760958: W ten

In [2]:
design = f3dasm.make_nd_continuous_design(bounds=np.tile([0.3,0.6], (3,1)), dimensionality=3)
sampler = f3dasm.sampling.SobolSequence(design=design, seed=42)
samples = sampler.get_samples(numsamples=20)


i = samples.to_json()

new_samples = f3dasm.design.create_experimentdata_from_json(i)

new_samples.data
sampler.__class__.__name__

'SobolSequence'

In [11]:
design.to_json()

'{"input_space": ["[{\\"name\\": \\"x0\\", \\"lower_bound\\": 0.3, \\"upper_bound\\": 0.6}, \\"ContinuousParameter\\"]", "[{\\"name\\": \\"x1\\", \\"lower_bound\\": 0.3, \\"upper_bound\\": 0.6}, \\"ContinuousParameter\\"]", "[{\\"name\\": \\"x2\\", \\"lower_bound\\": 0.3, \\"upper_bound\\": 0.6}, \\"ContinuousParameter\\"]"], "output_space": ["[{\\"name\\": \\"y\\", \\"lower_bound\\": -Infinity, \\"upper_bound\\": Infinity}, \\"ContinuousParameter\\"]"]}'

In [3]:
args = {'optimizer': f3dasm.optimization.RandomSearch(data=samples, seed=42),
        'function': f3dasm.functions.Ackley(dimensionality=3, scale_bounds=design.get_bounds()),
        'sampler': f3dasm.sampling.LatinHypercube(design=design, seed=42),
        'iterations': 1000,
        'realizations': 5,
        'number_of_samples': 30,
        'parallelization': True,
        'seed': 42
        }

optimizationresults: f3dasm.OptimizationResult = f3dasm.run_multiple_realizations(**args)

2023-02-27 14:20:10.330695: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-27 14:20:10.330794: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martin): /proc/driver/nvidia/version does not exist
2023-02-27 14:20:10.333030: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 14:20:18,361 - Optimized Ackley function (seed=None, dim=3, noise=None) with RandomSearch optimizer for 5 realizations!


In [7]:
json_results = optimizationresults.to_json()
with open('data1.json', 'w') as f:
    json.dump(json_results, f, ensure_ascii=False)

In [8]:
json_results

'{"data": ["{\\"design\\": \\"{\\\\\\"input_space\\\\\\": [\\\\\\"[{\\\\\\\\\\\\\\"name\\\\\\\\\\\\\\": \\\\\\\\\\\\\\"x0\\\\\\\\\\\\\\", \\\\\\\\\\\\\\"lower_bound\\\\\\\\\\\\\\": 0.3, \\\\\\\\\\\\\\"upper_bound\\\\\\\\\\\\\\": 0.6}, \\\\\\\\\\\\\\"ContinuousParameter\\\\\\\\\\\\\\"]\\\\\\", \\\\\\"[{\\\\\\\\\\\\\\"name\\\\\\\\\\\\\\": \\\\\\\\\\\\\\"x1\\\\\\\\\\\\\\", \\\\\\\\\\\\\\"lower_bound\\\\\\\\\\\\\\": 0.3, \\\\\\\\\\\\\\"upper_bound\\\\\\\\\\\\\\": 0.6}, \\\\\\\\\\\\\\"ContinuousParameter\\\\\\\\\\\\\\"]\\\\\\", \\\\\\"[{\\\\\\\\\\\\\\"name\\\\\\\\\\\\\\": \\\\\\\\\\\\\\"x2\\\\\\\\\\\\\\", \\\\\\\\\\\\\\"lower_bound\\\\\\\\\\\\\\": 0.3, \\\\\\\\\\\\\\"upper_bound\\\\\\\\\\\\\\": 0.6}, \\\\\\\\\\\\\\"ContinuousParameter\\\\\\\\\\\\\\"]\\\\\\"], \\\\\\"output_space\\\\\\": [\\\\\\"[{\\\\\\\\\\\\\\"name\\\\\\\\\\\\\\": \\\\\\\\\\\\\\"y\\\\\\\\\\\\\\", \\\\\\\\\\\\\\"lower_bound\\\\\\\\\\\\\\": -Infinity, \\\\\\\\\\\\\\"upper_bound\\\\\\\\\\\\\\": Infinity}, \\\\\\\\\\\\\\"Conti

In [ ]:
with open('data.json') as f:
    data = json.load(f)

In [ ]:
new_results = f3dasm.create_optimizationresult_from_json(data)

In [ ]:
new_optimizationresults = f3dasm.create_optimizationresult_from_json(json_results)

In [ ]:
new_optimizationresults == optimizationresults

In [ ]:
f3dasm.sampling.find_sampler('SobolSequence')

In [ ]:
sampler

In [ ]:
json_string = sampler.to_json()
json_string


In [ ]:
sampler_new = f3dasm.sampling.create_sampler_from_json(json_string)
sampler_new

In [ ]:
sampler == sampler_new

In [ ]:
all(samples.data == new_samples.data)

In [ ]:
f = f3dasm.functions.Ackley(dimensionality=3, scale_bounds=np.tile([-3.4,3.4], (3,1)),noise=0.3)

In [ ]:
f_json = f.to_json()
f_json

f3dasm.functions.create_function_from_json(f_json)

In [ ]:
optimizer = f3dasm.optimization.Adam(data=samples)
o_json = optimizer.to_json()
f3dasm.optimization.create_optimizer_from_json(o_json)

In [ ]:
def create_info(data):
    return {'design': data.design.get_info(),
    'data': data.data.to_json()}

In [ ]:
ii = samples.to_json()
json_data = json.dumps(ii)
json_data

In [ ]:
json_design = samples.design.get_info()
json_design

In [ ]:
# Read JSON
json_data_loaded = json.loads(json_data)

# Read design from json_data_loaded
new_design = f3dasm.design.create_design_from_dict(json_data_loaded['design'])

# Read data from json string
new_data = pd.read_json(json_data_loaded['data'])

# Create tuples of indices
columntuples = tuple(tuple(entry[1:-1].replace("'","").split(', ')) for entry in new_data.columns.values)

# Create MultiIndex object
columnlabels = pd.MultiIndex.from_tuples(columntuples)

# Overwrite columnlabels
new_data.columns = columnlabels

# Create
new_experimentdata = f3dasm.ExperimentData(design=new_design)
new_experimentdata.add(data=new_data)

In [ ]:
new_experimentdata.data

In [ ]:
# Create designspace from json file
def create_design_from_file(json_string):
    info = json.loads(json_string)

    for key, space in info.items():
        parameters = []
        for parameter in space:
            parameters.append(f3dasm.find_class(f3dasm.design, parameter[1])(**parameter[0]))

        info[key] = parameters

    design = f3dasm.DesignSpace(**info)
    return design

In [ ]:
info = design.get_info()
info

In [ ]:
i = json.dumps(design.get_info())
info = json.loads(i)

In [ ]:
f3dasm.optimization.OPTIMIZERS

In [ ]:
[opt.__class__.__name__ for opt in f3dasm.optimization.OPTIMIZERS]

In [ ]:
def find_optimizer(query: str):
    try:
        return list(filter(lambda x: x.get_name(x) == query, f3dasm.optimization.OPTIMIZERS))[0]
    except IndexError:
        return ValueError(f'Optimizer {query} not found!')

In [ ]:
find_optimizer('Adam')

In [ ]:
f3dasm.optimization.Adam(data=samples)

In [ ]:
optimizer = f3dasm.optimization.find_optimizer('Adam')(data=samples)

In [ ]:
info = optimizer.get_info()
opt_json = json.dumps(info)
new_optimizer = f3dasm.optimization.create_optimizer_from_json(opt_json)

In [ ]:
function = f3dasm.functions.Ackley(dimensionality=2, scale_bounds=np.tile([-3.,2.3], (2,1)), seed=42)

In [ ]:
info = function.get_info()
json_function = json.dumps(info)
json_function

In [ ]:
new_function = f3dasm.functions.create_function_from_json(json_function)

In [ ]:
new_function.augmentor == function.augmentor

In [ ]:
new_function.__dict__

In [ ]:
function.__dict__

In [ ]:
args = {'optimizer': optimizer,
        'function': function,
        'sampler': sampler,
        'iterations': 1000,
        'realizations': 5,
        'number_of_samples': 30,
        'parallelization': True,
        'seed': 42
        }

optimiztionresults = f3dasm.run_multiple_realizations(**args)